In [1]:
# ============================================================================
# Section 1: IMPORT VÀ CẤU HÌNH
# ============================================================================

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from sklearn.model_selection import train_test_split
import gc
import warnings
warnings.filterwarnings('ignore')

# Đường dẫn
TRAIN_SEQ_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta' 
TRAIN_TERMS_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv'
TEST_SEQ_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta'
OBO_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo'

# Hyperparameters
BATCH_SIZE = 64
EPOCHS = 20
LEARNING_RATE = 0.001
NUM_LABELS = 1500

# Ensemble config
SEEDS = [42, 123, 456, 789]  # 4 seeds khác nhau

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Device: {device}")
print(f"🎲 Ensemble với {len(SEEDS)} seeds: {SEEDS}")

🔧 Device: cuda
🎲 Ensemble với 4 seeds: [42, 123, 456, 789]


In [2]:
# ============================================================================
# Section 2: HELPER FUNCTIONS - LOAD DATA
# ============================================================================

def load_fasta(path):
    """Đọc file FASTA và trả về dict {protein_id: sequence}"""
    sequences = {}
    current_id = None
    current_seq = []
    
    with open(path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if current_id:
                    sequences[current_id] = ''.join(current_seq)
                
                header = line[1:]
                # Parse ID từ header
                if '|' in header:
                    parts = header.split('|')
                    current_id = parts[1] if len(parts) > 1 else header.split()[0]
                else:
                    current_id = header.split()[0]
                    
                current_seq = []
            else:
                current_seq.append(line)
                
        if current_id:
            sequences[current_id] = ''.join(current_seq)
            
    return sequences

def clean_sequence(seq):
    """
    Làm sạch chuỗi protein:
    - Chỉ giữ 20 amino acids chuẩn
    - Thay thế các ký tự đặc biệt
    """
    valid_aa = set('ACDEFGHIKLMNPQRSTVWY')
    replacements = {'X': 'A', 'U': 'C', 'B': 'D', 'Z': 'E'}
    
    cleaned = []
    for aa in seq.upper():
        if aa in valid_aa:
            cleaned.append(aa)
        elif aa in replacements:
            cleaned.append(replacements[aa])
    
    return ''.join(cleaned)

def is_valid_sequence(seq, min_len=50, max_len=5000):
    """Kiểm tra sequence có hợp lệ không"""
    length = len(seq)
    if length < min_len or length > max_len:
        return False
    
    valid_aa = set('ACDEFGHIKLMNPQRSTVWY')
    valid_count = sum(1 for aa in seq if aa in valid_aa)
    
    # Yêu cầu ít nhất 90% là amino acid chuẩn
    if valid_count / length < 0.9:
        return False
    
    return True

In [3]:
# ============================================================================
# Section 3: FEATURE EXTRACTION
# ============================================================================

def get_dipeptide_composition(sequence):
    """
    Tạo vector 400 chiều từ tần suất di-peptide
    Input: protein sequence
    Output: numpy array (400,)
    """
    aa_list = 'ACDEFGHIKLMNPQRSTVWY'
    aa_map = {aa: i for i, aa in enumerate(aa_list)}
    
    dipeptide_counts = np.zeros((20, 20), dtype=np.float32)
    length = len(sequence)
    
    if length < 2:
        return dipeptide_counts.flatten()
    
    # Đếm tần suất từng cặp amino acid
    for i in range(length - 1):
        a1 = sequence[i]
        a2 = sequence[i + 1]
        if a1 in aa_map and a2 in aa_map:
            dipeptide_counts[aa_map[a1], aa_map[a2]] += 1
    
    # Normalize bằng số cặp
    return dipeptide_counts.flatten() / (length - 1)


In [4]:
# ============================================================================
# Section 4: PREPROCESSING - LOAD VÀ CLEAN DATA
# ============================================================================

print("\n" + "="*70)
print("📂 BƯỚC 1: LOAD VÀ CLEAN DỮ LIỆU")
print("="*70)

# Load train sequences
print("\n1️⃣ Đang load train sequences...")
train_seqs_raw = load_fasta(TRAIN_SEQ_PATH)
print(f"   ✓ Loaded: {len(train_seqs_raw)} sequences")

# Clean sequences
print("\n2️⃣ Đang clean sequences...")
train_seqs = {}
invalid_count = 0

for pid, seq in train_seqs_raw.items():
    cleaned = clean_sequence(seq)
    if is_valid_sequence(cleaned):
        train_seqs[pid] = cleaned
    else:
        invalid_count += 1

print(f"   ✓ Valid sequences: {len(train_seqs)}")
print(f"   ✗ Invalid sequences: {invalid_count}")
print(f"   ✓ Keep ratio: {len(train_seqs)/len(train_seqs_raw)*100:.1f}%")

# Load train labels
print("\n3️⃣ Đang load train labels...")
train_terms = pd.read_csv(TRAIN_TERMS_PATH, sep='\t')
print(f"   ✓ Total annotations: {len(train_terms)}")
print(f"   ✓ Unique GO terms: {train_terms['term'].nunique()}")

# Filter top terms
print(f"\n4️⃣ Đang filter top {NUM_LABELS} GO terms...")
top_terms = train_terms['term'].value_counts().head(NUM_LABELS).index.tolist()
term_to_idx = {term: i for i, term in enumerate(top_terms)}
idx_to_term = {i: term for term, i in term_to_idx.items()}

train_terms_filtered = train_terms[
    (train_terms['term'].isin(top_terms)) &
    (train_terms['EntryID'].isin(train_seqs.keys()))
]

protein_to_labels = train_terms_filtered.groupby('EntryID')['term'].apply(list).to_dict()

# Get valid protein IDs (có cả sequence và labels)
valid_ids = [pid for pid in train_seqs.keys() if pid in protein_to_labels]
print(f"   ✓ Proteins với labels: {len(valid_ids)}")

# Statistics
seq_lengths = [len(train_seqs[pid]) for pid in valid_ids]
label_counts = [len(protein_to_labels[pid]) for pid in valid_ids]

print(f"\n📊 THỐNG KÊ:")
print(f"   Sequence length: min={min(seq_lengths)}, max={max(seq_lengths)}, mean={np.mean(seq_lengths):.0f}")
print(f"   Labels per protein: min={min(label_counts)}, max={max(label_counts)}, mean={np.mean(label_counts):.1f}")

# Free memory
del train_seqs_raw, train_terms, train_terms_filtered
gc.collect()


📂 BƯỚC 1: LOAD VÀ CLEAN DỮ LIỆU

1️⃣ Đang load train sequences...
   ✓ Loaded: 82404 sequences

2️⃣ Đang clean sequences...
   ✓ Valid sequences: 81417
   ✗ Invalid sequences: 987
   ✓ Keep ratio: 98.8%

3️⃣ Đang load train labels...
   ✓ Total annotations: 537027
   ✓ Unique GO terms: 26125

4️⃣ Đang filter top 1500 GO terms...
   ✓ Proteins với labels: 75371

📊 THỐNG KÊ:
   Sequence length: min=50, max=4998, mean=527
   Labels per protein: min=1, max=111, mean=4.5


0

In [5]:
# ============================================================================
# Section 5: FEATURE EXTRACTION CHO TOÀN BỘ DATASET
# ============================================================================

print("\n" + "="*70)
print("🔧 BƯỚC 2: FEATURE EXTRACTION")
print("="*70)

print("\n🔨 Đang extract dipeptide features cho tất cả proteins...")
all_features = {}

for i, pid in enumerate(valid_ids):
    seq = train_seqs[pid]
    features = get_dipeptide_composition(seq)
    all_features[pid] = features
    
    if (i + 1) % 5000 == 0:
        print(f"   Progress: {i+1}/{len(valid_ids)} ({(i+1)/len(valid_ids)*100:.1f}%)")

print(f"\n✓ Completed: {len(all_features)} feature vectors (400 dims each)")

# Verify
sample_pid = valid_ids[0]
print(f"\n📋 Sample check:")
print(f"   Protein ID: {sample_pid}")
print(f"   Sequence length: {len(train_seqs[sample_pid])}")
print(f"   Feature shape: {all_features[sample_pid].shape}")
print(f"   Feature range: [{all_features[sample_pid].min():.4f}, {all_features[sample_pid].max():.4f}]")


🔧 BƯỚC 2: FEATURE EXTRACTION

🔨 Đang extract dipeptide features cho tất cả proteins...
   Progress: 5000/75371 (6.6%)
   Progress: 10000/75371 (13.3%)
   Progress: 15000/75371 (19.9%)
   Progress: 20000/75371 (26.5%)
   Progress: 25000/75371 (33.2%)
   Progress: 30000/75371 (39.8%)
   Progress: 35000/75371 (46.4%)
   Progress: 40000/75371 (53.1%)
   Progress: 45000/75371 (59.7%)
   Progress: 50000/75371 (66.3%)
   Progress: 55000/75371 (73.0%)
   Progress: 60000/75371 (79.6%)
   Progress: 65000/75371 (86.2%)
   Progress: 70000/75371 (92.9%)
   Progress: 75000/75371 (99.5%)

✓ Completed: 75371 feature vectors (400 dims each)

📋 Sample check:
   Protein ID: A0JNW5
   Sequence length: 1464
   Feature shape: (400,)
   Feature range: [0.0000, 0.0150]


In [6]:
# ============================================================================
# Section 6: PYTORCH DATASET CLASS
# ============================================================================

class CAFA6Dataset(Dataset):
    """PyTorch Dataset cho CAFA-6"""
    def __init__(self, protein_ids, features_dict, label_dict, term_map, num_classes):
        self.protein_ids = protein_ids
        self.features_dict = features_dict
        self.label_dict = label_dict
        self.term_map = term_map
        self.num_classes = num_classes
    
    def __len__(self):
        return len(self.protein_ids)
    
    def __getitem__(self, idx):
        pid = self.protein_ids[idx]
        
        # Features
        features = self.features_dict[pid]
        
        # Labels (multi-hot encoding)
        labels = np.zeros(self.num_classes, dtype=np.float32)
        if pid in self.label_dict:
            for term in self.label_dict[pid]:
                if term in self.term_map:
                    labels[self.term_map[term]] = 1.0
        
        return (
            torch.tensor(features, dtype=torch.float32),
            torch.tensor(labels, dtype=torch.float32)
        )

In [7]:
# ============================================================================
# Section 7: MODEL ARCHITECTURE
# ============================================================================

class ResidualBlock(nn.Module):
    """Residual block với skip connection"""
    def __init__(self, dim, dropout_rate=0.4):
        super(ResidualBlock, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
    
    def forward(self, x):
        return x + self.layer(x)

class ResMLP(nn.Module):
    """Residual MLP cho multi-label classification"""
    def __init__(self, input_dim=400, num_classes=1500):
        super(ResMLP, self).__init__()
        
        self.entry = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        self.blocks = nn.Sequential(
            ResidualBlock(1024, dropout_rate=0.4),
            ResidualBlock(1024, dropout_rate=0.4)
        )
        
        self.head = nn.Linear(1024, num_classes)
    
    def forward(self, x):
        x = self.entry(x)
        x = self.blocks(x)
        return self.head(x)

In [8]:
# ============================================================================
# Section 8: TRAINING FUNCTION
# ============================================================================

def train_single_seed(seed, train_ids, val_ids, all_features, protein_to_labels, 
                      term_to_idx, save_path):
    """
    Train model với 1 seed cụ thể
    
    Args:
        seed: random seed
        train_ids: list protein IDs cho training
        val_ids: list protein IDs cho validation
        all_features: dict features
        protein_to_labels: dict labels
        term_to_idx: dict term mapping
        save_path: path lưu model
    
    Returns:
        best_val_loss: validation loss tốt nhất
    """
    print(f"\n{'='*70}")
    print(f"🎲 TRAINING SEED: {seed}")
    print(f"{'='*70}")
    
    # Set seed
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    
    # Create datasets
    train_dataset = CAFA6Dataset(
        train_ids, all_features, protein_to_labels, term_to_idx, NUM_LABELS
    )
    val_dataset = CAFA6Dataset(
        val_ids, all_features, protein_to_labels, term_to_idx, NUM_LABELS
    )
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    # Create model
    model = ResMLP(input_dim=400, num_classes=NUM_LABELS).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=3, verbose=False
    )
    
    best_val_loss = float('inf')
    patience_counter = 0
    
    # Training loop
    for epoch in range(EPOCHS):
        # Train
        model.train()
        train_loss = 0.0
        
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        avg_train_loss = train_loss / len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_loader)
        
        # Scheduler step
        scheduler.step(avg_val_loss)
        
        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), save_path)
            patience_counter = 0
        else:
            patience_counter += 1
        
        # Print progress
        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f"   Epoch {epoch+1:2d}/{EPOCHS} | "
                  f"Train: {avg_train_loss:.4f} | "
                  f"Val: {avg_val_loss:.4f} | "
                  f"Best: {best_val_loss:.4f}")
    
    print(f"\n✓ Seed {seed} completed. Best val loss: {best_val_loss:.4f}")
    print(f"✓ Model saved to: {save_path}")
    
    return best_val_loss

In [9]:
# ============================================================================
# Section 9: TRAIN ENSEMBLE (4 SEEDS)
# ============================================================================

print("\n" + "="*70)
print("🚀 BƯỚC 3: TRAINING ENSEMBLE (4 SEEDS)")
print("="*70)

# Tạo thư mục lưu models
os.makedirs('models', exist_ok=True)

model_paths = []
val_losses = []

for seed in SEEDS:
    # Split data với seed này
    train_ids, val_ids = train_test_split(
        valid_ids, test_size=0.1, random_state=seed
    )
    
    print(f"\n📊 Seed {seed}: Train={len(train_ids)}, Val={len(val_ids)}")
    
    # Train
    save_path = f'models/model_seed_{seed}.pth'
    val_loss = train_single_seed(
        seed, train_ids, val_ids, all_features, 
        protein_to_labels, term_to_idx, save_path
    )
    
    model_paths.append(save_path)
    val_losses.append(val_loss)
    
    # Free memory
    torch.cuda.empty_cache()
    gc.collect()

# Summary
print("\n" + "="*70)
print("📈 ENSEMBLE TRAINING SUMMARY")
print("="*70)
for seed, loss in zip(SEEDS, val_losses):
    print(f"   Seed {seed}: Val Loss = {loss:.4f}")
print(f"\n   Mean Val Loss: {np.mean(val_losses):.4f}")
print(f"   Std Val Loss:  {np.std(val_losses):.4f}")


🚀 BƯỚC 3: TRAINING ENSEMBLE (4 SEEDS)

📊 Seed 42: Train=67833, Val=7538

🎲 TRAINING SEED: 42
   Epoch  1/20 | Train: 0.0175 | Val: 0.0151 | Best: 0.0151
   Epoch  5/20 | Train: 0.0129 | Val: 0.0140 | Best: 0.0140
   Epoch 10/20 | Train: 0.0109 | Val: 0.0142 | Best: 0.0140
   Epoch 15/20 | Train: 0.0090 | Val: 0.0145 | Best: 0.0140
   Epoch 20/20 | Train: 0.0084 | Val: 0.0147 | Best: 0.0140

✓ Seed 42 completed. Best val loss: 0.0140
✓ Model saved to: models/model_seed_42.pth

📊 Seed 123: Train=67833, Val=7538

🎲 TRAINING SEED: 123
   Epoch  1/20 | Train: 0.0175 | Val: 0.0151 | Best: 0.0151
   Epoch  5/20 | Train: 0.0129 | Val: 0.0140 | Best: 0.0140
   Epoch 10/20 | Train: 0.0109 | Val: 0.0142 | Best: 0.0140
   Epoch 15/20 | Train: 0.0090 | Val: 0.0145 | Best: 0.0140
   Epoch 20/20 | Train: 0.0084 | Val: 0.0147 | Best: 0.0140

✓ Seed 123 completed. Best val loss: 0.0140
✓ Model saved to: models/model_seed_123.pth

📊 Seed 456: Train=67833, Val=7538

🎲 TRAINING SEED: 456
   Epoch  1/20 |

In [10]:
# ============================================================================
# Section 10: LOAD TEST DATA VÀ EXTRACT FEATURES
# ============================================================================

print("\n" + "="*70)
print("📂 BƯỚC 4: LOAD VÀ PREPROCESS TEST DATA")
print("="*70)

print("\n1️⃣ Đang load test sequences...")
test_seqs_raw = load_fasta(TEST_SEQ_PATH)
print(f"   ✓ Loaded: {len(test_seqs_raw)} sequences")

# Clean test sequences
print("\n2️⃣ Đang clean test sequences...")
test_seqs = {}
for pid, seq in test_seqs_raw.items():
    cleaned = clean_sequence(seq)
    if is_valid_sequence(cleaned, min_len=10):  # Lỏng hơn cho test
        test_seqs[pid] = cleaned

print(f"   ✓ Valid test sequences: {len(test_seqs)}")

# Extract features
print("\n3️⃣ Đang extract features cho test set...")
test_features = {}

for i, (pid, seq) in enumerate(test_seqs.items()):
    features = get_dipeptide_composition(seq)
    test_features[pid] = features
    
    if (i + 1) % 5000 == 0:
        print(f"   Progress: {i+1}/{len(test_seqs)} ({(i+1)/len(test_seqs)*100:.1f}%)")

print(f"\n✓ Test features ready: {len(test_features)} proteins")

# Free memory
del test_seqs_raw
gc.collect()


📂 BƯỚC 4: LOAD VÀ PREPROCESS TEST DATA

1️⃣ Đang load test sequences...
   ✓ Loaded: 224309 sequences

2️⃣ Đang clean test sequences...
   ✓ Valid test sequences: 223371

3️⃣ Đang extract features cho test set...
   Progress: 5000/223371 (2.2%)
   Progress: 10000/223371 (4.5%)
   Progress: 15000/223371 (6.7%)
   Progress: 20000/223371 (9.0%)
   Progress: 25000/223371 (11.2%)
   Progress: 30000/223371 (13.4%)
   Progress: 35000/223371 (15.7%)
   Progress: 40000/223371 (17.9%)
   Progress: 45000/223371 (20.1%)
   Progress: 50000/223371 (22.4%)
   Progress: 55000/223371 (24.6%)
   Progress: 60000/223371 (26.9%)
   Progress: 65000/223371 (29.1%)
   Progress: 70000/223371 (31.3%)
   Progress: 75000/223371 (33.6%)
   Progress: 80000/223371 (35.8%)
   Progress: 85000/223371 (38.1%)
   Progress: 90000/223371 (40.3%)
   Progress: 95000/223371 (42.5%)
   Progress: 100000/223371 (44.8%)
   Progress: 105000/223371 (47.0%)
   Progress: 110000/223371 (49.2%)
   Progress: 115000/223371 (51.5%)
   Pr

0

In [11]:
# ============================================================================
# Section 11: ENSEMBLE PREDICTION
# ============================================================================

print("\n" + "="*70)
print("🔮 BƯỚC 5: ENSEMBLE PREDICTION")
print("="*70)

# Load all models
models = []
print("\n📥 Loading models...")
for i, path in enumerate(model_paths):
    model = ResMLP(input_dim=400, num_classes=NUM_LABELS).to(device)
    model.load_state_dict(torch.load(path))
    model.eval()
    models.append(model)
    print(f"   ✓ Model {i+1}/{len(models)} loaded from {path}")

# Predict với ensemble
print("\n🔮 Đang dự đoán với ensemble...")
TEMP_SUBMISSION_FILE = 'submission_temp.tsv'

with open(TEMP_SUBMISSION_FILE, 'w') as f:
    for count, (pid, seq) in enumerate(test_seqs.items(), 1):
        features = test_features[pid]
        features_tensor = torch.tensor([features], dtype=torch.float32).to(device)
        
        # Ensemble: Average predictions từ 4 models
        ensemble_probs = np.zeros(NUM_LABELS, dtype=np.float32)
        
        with torch.no_grad():
            for model in models:
                logits = model(features_tensor)
                probs = torch.sigmoid(logits).cpu().numpy()[0]
                ensemble_probs += probs
        
        # Average
        ensemble_probs /= len(models)
        
        # Lấy top 50 predictions
        top_indices = np.argsort(ensemble_probs)[::-1][:50]
        
        for idx in top_indices:
            score = ensemble_probs[idx]
            if score > 0.005:
                term_id = idx_to_term[idx]
                f.write(f"{pid}\t{term_id}\t{score:.3f}\n")
        
        if count % 2000 == 0:
            print(f"   Progress: {count}/{len(test_seqs)} proteins")

print(f"\n✓ Predictions saved to: {TEMP_SUBMISSION_FILE}")


🔮 BƯỚC 5: ENSEMBLE PREDICTION

📥 Loading models...
   ✓ Model 1/1 loaded from models/model_seed_42.pth
   ✓ Model 2/2 loaded from models/model_seed_123.pth
   ✓ Model 3/3 loaded from models/model_seed_456.pth
   ✓ Model 4/4 loaded from models/model_seed_789.pth

🔮 Đang dự đoán với ensemble...
   Progress: 2000/223371 proteins
   Progress: 4000/223371 proteins
   Progress: 6000/223371 proteins
   Progress: 8000/223371 proteins
   Progress: 10000/223371 proteins
   Progress: 12000/223371 proteins
   Progress: 14000/223371 proteins
   Progress: 16000/223371 proteins
   Progress: 18000/223371 proteins
   Progress: 20000/223371 proteins
   Progress: 22000/223371 proteins
   Progress: 24000/223371 proteins
   Progress: 26000/223371 proteins
   Progress: 28000/223371 proteins
   Progress: 30000/223371 proteins
   Progress: 32000/223371 proteins
   Progress: 34000/223371 proteins
   Progress: 36000/223371 proteins
   Progress: 38000/223371 proteins
   Progress: 40000/223371 proteins
   Progre

In [12]:
# ============================================================================
# Section 12: POST-PROCESSING (GO HIERARCHY PROPAGATION)
# ============================================================================

print("\n" + "="*70)
print("🌳 BƯỚC 6: POST-PROCESSING (GO Hierarchy)")
print("="*70)

FINAL_OUTPUT = 'submission.tsv'

# Install goatools nếu cần
try:
    from goatools.obo_parser import GODag
except ImportError:
    print("📦 Installing goatools...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "goatools", "-q"])
    from goatools.obo_parser import GODag

if not os.path.exists(OBO_PATH):
    print("⚠️  OBO file not found. Using raw predictions.")
    os.rename(TEMP_SUBMISSION_FILE, FINAL_OUTPUT)
else:
    print("\n1️⃣ Loading GO hierarchy...")
    godag = GODag(OBO_PATH)
    print(f"   ✓ Loaded {len(godag)} GO terms")
    
    def propagate_scores(df_group, godag):
        """Propagate scores lên ancestor terms"""
        current_scores = dict(zip(df_group['GO_Term'], df_group['Score']))
        new_scores = current_scores.copy()
        
        for go_id, score in current_scores.items():
            if go_id not in godag:
                continue
            
            term_obj = godag[go_id]
            ancestors = term_obj.get_all_parents()
            
            # Parent phải có score >= child
            for ancestor in ancestors:
                anc_score_old = new_scores.get(ancestor, 0.0)
                new_scores[ancestor] = max(anc_score_old, score)
        
        return [[pid, term, score] for term, score in new_scores.items() if score >= 0.01]
    
    # Load temp submission
    print("\n2️⃣ Loading predictions...")
    sub_df = pd.read_csv(TEMP_SUBMISSION_FILE, sep='\t', 
                         names=['ProteinID', 'GO_Term', 'Score'])
    print(f"   ✓ Loaded {len(sub_df)} predictions")
    
    # Propagate
    print("\n3️⃣ Propagating scores...")
    final_data = []
    
    try:
        from tqdm import tqdm
        iterator = tqdm(sub_df.groupby('ProteinID'), desc="   Proteins")
    except ImportError:
        iterator = sub_df.groupby('ProteinID')
        print("   (Processing without progress bar)")
    
    for pid, group in iterator:
        refined_rows = propagate_scores(group, godag)
        final_data.extend(refined_rows)
    
    # Save final submission
    print("\n4️⃣ Saving final submission...")
    result_df = pd.DataFrame(final_data, columns=['ProteinID', 'GO_Term', 'Score'])
    result_df['Score'] = result_df['Score'].map(lambda x: f'{x:.3f}')
    result_df.to_csv(FINAL_OUTPUT, sep='\t', index=False, header=False)
    
    print(f"   ✓ Total predictions: {len(result_df)}")
    print(f"   ✓ Unique proteins: {result_df['ProteinID'].nunique()}")
    print(f"   ✓ Unique GO terms: {result_df['GO_Term'].nunique()}")

print("\n" + "="*70)
print(f"✅ HOÀN TẤT! File submission: {FINAL_OUTPUT}")
print("="*70)


🌳 BƯỚC 6: POST-PROCESSING (GO Hierarchy)
📦 Installing goatools...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 8.6 MB/s eta 0:00:00

1️⃣ Loading GO hierarchy...
/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo: fmt(1.2) rel(2025-06-01) 43,448 Terms
   ✓ Loaded 43448 GO terms

2️⃣ Loading predictions...
   ✓ Loaded 10266484 predictions

3️⃣ Propagating scores...


   Proteins: 100%|██████████| 223371/223371 [01:59<00:00, 1866.11it/s]



4️⃣ Saving final submission...
   ✓ Total predictions: 28752156
   ✓ Unique proteins: 223369
   ✓ Unique GO terms: 2935

✅ HOÀN TẤT! File submission: submission.tsv
